# OrthodoxAI - LLM Chains

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros?tab=repositories<br>
> Year: 2025

## About the Project

## Libraries

In [1]:
# Load the API Keys
import os
from dotenv import load_dotenv
_ = load_dotenv()

from pydantic import BaseModel, Field
from typing import List, Literal

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

## Prompt Templates

In [44]:
class AnalyzerOutput(BaseModel):
    classification: Literal["Religious", "Non-Religious"] = Field(..., description="Either 'Religious' or 'Non-Religious'")
    key_topics: List[str] = Field(..., description="List of key topics/areas related to the user's question (e.g., theology, jesus, humility, virtues)",)
    context_requirements: str = Field(..., description="A clear explanation of the query's context needs")
    query_complexity: Literal["Low", "Medium", "High"] = Field(..., description="'Low', 'Medium', or 'High' complexity")
    reasoning: str = Field(..., description="The Chain of Thought that has been done in order to analyze the user query")

In [45]:
class SummarizerOutput(BaseModel):
        summary: str = Field(..., description="")

## LLM Chains

### Analyzer Agent

In [69]:
analyzer_llm = ChatOpenAI(model="o3-mini").with_structured_output(AnalyzerOutput)

In [70]:
analyzer_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an AI assistant that classifies user queries as either religious or non-religious and extracts key topics.",
            ),
            (
                "human",
                "Classify the following query and extract key concepts:\n\nQuery: {query}",
            ),
        ]
    )

In [71]:
analyzer_chain = analyzer_prompt | analyzer_llm

In [72]:
text = analyzer_chain.invoke('Hello')
text

AnalyzerOutput(classification='Non-Religious', key_topics=['greeting'], context_requirements='No additional context is provided beyond a basic greeting.', query_complexity='Low', reasoning="The query 'Hello' is a simple greeting without any religious connotations. It is non-religious in nature and does not contain hints of theological or religious content, so it is classified as Non-Religious with key focus on greeting.")

In [73]:
print(text.classification)
print(text.context_requirements)
print(text.key_topics)
print(text.query_complexity)
print(text.reasoning)

Non-Religious
No additional context is provided beyond a basic greeting.
['greeting']
Low
The query 'Hello' is a simple greeting without any religious connotations. It is non-religious in nature and does not contain hints of theological or religious content, so it is classified as Non-Religious with key focus on greeting.


In [74]:
type(text)

__main__.AnalyzerOutput

In [75]:
analysis_report = (
    f"The analysis of the user query shows that the question is {text.classification} "
    f"and with a {text.query_complexity} complexity. Specifically, the analysis said the following: "
    f"{text.context_requirements} Key topics: {', '.join(text.key_topics)}. Chain of Thought: {text.reasoning}"
)
print(analysis_report)

The analysis of the user query shows that the question is Non-Religious and with a Low complexity. Specifically, the analysis said the following: No additional context is provided beyond a basic greeting. Key topics: greeting. Chain of Thought: The query 'Hello' is a simple greeting without any religious connotations. It is non-religious in nature and does not contain hints of theological or religious content, so it is classified as Non-Religious with key focus on greeting.


### Summarizer Agent